В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов 

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/LexusShabunya/projects/main/DATA/ds_salaries.csv")

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1

Удалите из признаков (salary) чтобы избежать лика в данных.

In [4]:
from sklearn.model_selection import train_test_split


X = df.drop(columns=['salary_in_usd', 'salary'])
y = df['salary_in_usd']

X.sample(5)

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
2624,2022,SE,FT,Data Analyst,USD,US,0,US,M
3027,2022,SE,FT,Data Scientist,USD,US,0,US,M
1587,2023,SE,FT,Data Architect,USD,US,0,US,M
1487,2023,SE,FT,Data Scientist,USD,US,100,US,M
2774,2022,SE,FT,ML Engineer,USD,US,100,US,M


## Задание 2  

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [5]:
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder(drop='first')

X_normalized = enc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error


lm = LinearRegression()

lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)


print(f'MAPE train: {mean_absolute_percentage_error(y_train, lm.predict(X_train))}')
print(f'RMSE train: {mean_squared_error(y_train, lm.predict(X_train))}')
print(f'MAPE test: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'RMSE test: {mean_squared_error(y_test, y_pred)}')

# видно что модель довольно посредственная

MAPE train: 0.30243038762867247
RMSE train: 1995264825.7857862
MAPE test: 0.4112197329166073
RMSE test: 2447455262.5580535


## Задание 3  

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from catboost import CatBoostRegressor


cat_gb = CatBoostRegressor(cat_features=[1, 2, 3, 4, 5, 7, 8], silent=True)

cat_gb.fit(X_train, y_train)

y_pred = cat_gb.predict(X_test)


print(f'MAPE train: {mean_absolute_percentage_error(y_train, cat_gb.predict(X_train))}')
print(f'RMSE train: {mean_squared_error(y_train, cat_gb.predict(X_train))}')
print(f'MAPE test: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'RMSE test: {mean_squared_error(y_test, y_pred)}')

# эта модель не лучше линейной

MAPE train: 0.30530898114188787
RMSE train: 1979359719.7715623
MAPE test: 0.3718019599308132
RMSE test: 2303287028.506103


## Задание 4  

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

**Закодируем признаки**

In [9]:
ordinal_features = ['experience_level', 'company_size'] # закодируем лэйбел энкодингом

ohe_features = ['employment_type'] # закодируем ван хот энкодингом

mte_features = ['job_title', 'salary_currency', 'employee_residence', 'company_location'] # закодируем таргет энкодингом

In [10]:
# таблица соответствий
ord_table = {
    'experience_level': {'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}, 
    'company_size': {'S':0, 'M': 1, 'L': 2}
}


X = X.replace(ord_table) # кодируем порядковые признаки
X.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2023,2,FT,Principal Data Scientist,EUR,ES,100,ES,2
1,2023,1,CT,ML Engineer,USD,US,100,US,0
2,2023,1,CT,ML Engineer,USD,US,100,US,0
3,2023,2,FT,Data Scientist,USD,CA,100,CA,1
4,2023,2,FT,Data Scientist,USD,CA,100,CA,1


In [11]:
enc = OneHotEncoder(drop='first', sparse_output=False)

dum_features = enc.fit_transform(X[ohe_features])
dum_features = pd.DataFrame(dum_features, columns=enc.get_feature_names_out())


X = pd.concat([X.drop(columns=ohe_features), dum_features], axis=1)
X.head()

,work_year,experience_level,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,employment_type_FL,employment_type_FT,employment_type_PT
0,2023,2,Principal Data Scientist,EUR,ES,100,ES,2,0.0,1.0,0.0
1,2023,1,ML Engineer,USD,US,100,US,0,0.0,0.0,0.0
2,2023,1,ML Engineer,USD,US,100,US,0,0.0,0.0,0.0
3,2023,2,Data Scientist,USD,CA,100,CA,1,0.0,1.0,0.0
4,2023,2,Data Scientist,USD,CA,100,CA,1,0.0,1.0,0.0


In [12]:
from sklearn.preprocessing import TargetEncoder


enc = TargetEncoder(random_state=42, target_type='continuous')

mte_data = enc.fit_transform(X[mte_features], y)

mte_data = pd.DataFrame(mte_data, columns=enc.get_feature_names_out())


X = pd.concat([X.drop(columns=mte_features), mte_data], axis=1)
X.head()

,work_year,experience_level,remote_ratio,company_size,employment_type_FL,employment_type_FT,employment_type_PT,job_title,salary_currency,employee_residence,company_location
0,2023,2,100,2,0.0,1.0,0.0,197557.256736,62291.174973,63794.455817,61324.836040
1,2023,1,100,0,0.0,0.0,0.0,160217.678112,149919.033516,153326.292099,152573.845460
2,2023,1,100,0,0.0,0.0,0.0,159767.201851,148798.902690,152599.119475,151570.271383
3,2023,2,100,1,0.0,1.0,0.0,140501.521845,149253.353938,135095.078137,134159.536955
4,2023,2,100,1,0.0,1.0,0.0,142802.032293,149919.033516,128440.657649,129192.882320


**Обучим и сравним модели**

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [18]:
lm = LinearRegression()

gb = CatBoostRegressor(silent=True)

for model in ((lm, 'Linear Regression'), (gb, 'Boosting')):
    model[0].fit(X_train, y_train)
    
    y_pred = model[0].predict(X_val)
    
    print(f'{model[1]} MAPE train: {mean_absolute_percentage_error(y_train, model[0].predict(X_train))}')
    print(f'{model[1]} RMSE train: {mean_squared_error(y_train, model[0].predict(X_train))}')
    print(f'{model[1]} MAPE test: {mean_absolute_percentage_error(y_val, y_pred)}')
    print(f'{model[1]} RMSE test: {mean_squared_error(y_val, y_pred)}')
    

# в базовом варианте линейная регрессия показывает себя лучше

Linear Regression MAPE train: 0.3783179189343234
Linear Regression RMSE train: 2392205961.575519
Linear Regression MAPE test: 0.3965341581894166
Linear Regression RMSE test: 2107197461.4792514
Boosting MAPE train: 0.255588297501506
Boosting RMSE train: 1610223891.441282
Boosting MAPE test: 0.39332853237866355
Boosting RMSE test: 2244866101.0709953


## Задание 5  

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [40]:
losses = np.sqrt(np.square(y_val - lm.predict(X_val)))

highest_losses = losses.sort_values(ascending=False)[:20].index

df.iloc[highest_losses]

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
1105,2023,SE,FT,Data Scientist,370000,USD,370000,US,0,US,M
478,2023,EX,FT,Director of Data Science,353200,USD,353200,US,0,US,M
3609,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
358,2023,SE,FT,Machine Learning Engineer,304000,USD,304000,US,100,US,M
3477,2022,EN,FT,Computer Vision Software Engineer,150000,USD,150000,AU,100,AU,S
3387,2022,SE,FT,Data Engineer,25000,USD,25000,US,100,US,M
2009,2022,SE,FT,Machine Learning Engineer,255000,USD,255000,MX,100,MX,M
549,2023,SE,FT,Data Scientist,275300,USD,275300,US,100,US,M
2281,2022,SE,FT,Data Science Manager,299500,USD,299500,US,0,US,M
1425,2023,SE,FT,Machine Learning Engineer,276000,USD,276000,US,0,US,M


Видно что наибольшая ошибка у модели на объектах которые получают ЗП в долларах. Кроме того, модель сильно ошибается на работниках которые и работают на компанию из США, и являются резидентами США. На SE спецалистах модель тоже сильно ошибается.  

Попробуем добавить признаки в модель.

## Задание 6  

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Важно попробовать проверить свои гипотезы и сделать выводы

## Задание 7  

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

## Задание 8  

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 9  

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 10  

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания